# Prepare Production reaction structure pictures
> author: Shizhenkun   
> email: zhenkun.shi@tib.cas.cn   
> date: 2024-09-11  



## 1. Import packages

In [1]:
import sys,os
sys.path.insert(0, os.path.dirname(os.path.realpath('__file__')))
sys.path.insert(1,'../')
from config import conf as cfg
from modules.rxn.Reaction import Reaction
from modules.rxn.Molecule import Molecule
from tools import  s3tool as s3tool
from tqdm import tqdm
from IPython.display import SVG, HTML, display
import pandas as pd
tqdm.pandas()  # Register the pandas integration with tqdm
FIRST_TIME_RUN = False # For the initial run, please set this flag to True. This will allow the program to download data from UniProt and RHEA, which may take longer depending on your internet speed.

%load_ext autoreload
%autoreload 2

## 2. Load data and Make reaction structure picture

In [7]:
ds_rxn = pd.read_feather(cfg.FILE_DS_RHEA_REACTIONS)
ds_rxn.head(2)

,reaction_id,equation,chebi_id,ec_number,equation_chebi,equation_smiles
0,RHEA:22636,dCTP + H2O = dCMP + diphosphate + H(+),CHEBI:61481;CHEBI:15377;CHEBI:57566;CHEBI:3301...,EC:3.6.1.9;EC:3.6.1.12;EC:3.6.1.65,CHEBI:61481 + CHEBI:15377 = CHEBI:57566 + CHEB...,Nc1ccn([C@H]2C[C@H](O)[C@@H](COP([O-])(=O)OP([...
1,RHEA:22640,NADP(+) + sphinganine = 3-oxosphinganine + H(+...,CHEBI:58349;CHEBI:57817;CHEBI:58299;CHEBI:1537...,EC:1.1.1.102,CHEBI:58349 + CHEBI:57817 = CHEBI:58299 + CHEB...,NC(=O)c1ccc[n+](c1)[C@@H]1O[C@H](COP([O-])(=O)...


## 3.Make JSON file for reaction structure visualization

In [8]:
def make_rxn_structure_json(row):
    
    rxn_id = row.reaction_id
    rxn_smiles = row.equation_smiles
    rxn_equation = row.equation
    rxn_equation_ref_chebi = row.equation_chebi
    file_path = f'{cfg.RXN_JSON_DIR}{rxn_id}.json'.replace(':','_')
    write_path = f'{cfg.DIR_PROJECT_ROOT}/{file_path}'
    
    if not os.path.exists(write_path):
        rxn = Reaction( rxn_id=rxn_id,
                        rxn_smiles=rxn_smiles,
                        rxn_equation=rxn_equation,
                        rxn_equation_ref_chebi=rxn_equation_ref_chebi
                        )

        rxn.save_json_file(file_path=write_path)
        s3tool.upload_file_to_s3(file=write_path, file_key=file_path, format='json')
        
    return file_path

In [11]:
ds_rxn.progress_apply(lambda x: make_rxn_structure_json(x), axis=1)

100%|██████████| 12198/12198 [00:04<00:00, 2735.19it/s]


0        production/files/json/RHEA_22636.json
1        production/files/json/RHEA_22640.json
2        production/files/json/RHEA_22644.json
3        production/files/json/RHEA_22648.json
4        production/files/json/RHEA_22652.json
                         ...                  
12193    production/files/json/RHEA_22604.json
12194    production/files/json/RHEA_22608.json
12195    production/files/json/RHEA_22612.json
12196    production/files/json/RHEA_22624.json
12197    production/files/json/RHEA_22628.json
Length: 12198, dtype: object

### 反应作图示例

In [12]:
rxn_id = 'RHEA:75967'
rxn1 = Reaction(rxn_smiles=ds_rxn[ds_rxn.reaction_id == rxn_id].equation_smiles.values[0],
                rxn_equation=ds_rxn[ds_rxn.reaction_id == rxn_id].equation.values[0],
                rxn_equation_ref_chebi=ds_rxn[ds_rxn.reaction_id == rxn_id].equation_chebi.values[0]
                )
HTML(rxn1.to_html())

### 化合物作图示例

In [14]:
mol1 =  Molecule(cpd_smiles=ds_rxn[ds_rxn.reaction_id == 'RHEA:22648'].equation_smiles.values[0].split('>>')[0], 
                 cpd_name=ds_rxn[ds_rxn.reaction_id == 'RHEA:22648'].equation.values[0].split('+')[0],
                 cpd_ref_chebi='CHEBI:15377'
                 )
mol1.write_mol_svg(cfg.TEMP_DIR + "mol1.svg")       
HTML(mol1.to_html())